
# 🧾 Budget Tracker Notebook: Full Process

This notebook handles the full backend logic of the budget tracker:
- Upload raw transaction CSVs/XLSX
- Clean and normalize them
- Apply categorization using a local JSON dictionary
- Output cleaned expenses
- Compare actual vs budget

---

## 📦 1. Setup

```python
import pandas as pd
import json
from datetime import datetime
from pathlib import Path
```

---

## 📁 2. Upload Files

```python
# Define file paths
csv_paths = [
    'data/santander.csv',
    'data/bbva.xlsx',
    'data/amex.xlsx'
]

# Load files into DataFrames
raw_transactions = []

for path in csv_paths:
    ext = Path(path).suffix
    if ext == '.csv':
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)
    df['source'] = Path(path).stem
    raw_transactions.append(df)

# Combine into one DataFrame
df_raw = pd.concat(raw_transactions, ignore_index=True)
df_raw.head()
```

---

## 🧹 3. Normalize and Clean Data

```python
# Try to map typical column names
def normalize_df(df):
    col_map = {}
    for col in df.columns:
        name = col.lower()
        if 'fecha' in name or 'date' in name:
            col_map[col] = 'date'
        elif 'descripcion' in name or 'description' in name or 'concepto' in name:
            col_map[col] = 'description'
        elif 'monto' in name or 'amount' in name or 'cargo' in name or 'retiro' in name:
            col_map[col] = 'amount'
    df = df.rename(columns=col_map)
    df = df[['date', 'description', 'amount', 'source']]
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date', 'amount'])
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce').abs()
    df['tag'] = df['description'].str.lower().str.strip().str[:12]
    return df

df_cleaned = normalize_df(df_raw)
df_cleaned.head()
```

---

## 🧠 4. Apply Categorization (JSON Dictionary)

```python
# Load your category dictionary
with open('data/categories.json', 'r') as f:
    tag_map = json.load(f)

df_cleaned['category'] = df_cleaned['tag'].map(tag_map).fillna('uncategorized')
df_cleaned.head()
```

---

## 📊 5. Load Budget and Compare vs Actuals

```python
# Load budget CSV with columns: category, budget
df_budget = pd.read_csv('data/budget.csv')
df_budget.columns = ['category', 'budget']
df_budget['category'] = df_budget['category'].str.strip()

# Group actual expenses
df_actuals = df_cleaned.groupby('category', as_index=False)['amount'].sum()
df_actuals = df_actuals.rename(columns={'amount': 'actual_spent'})

# Merge and calculate difference
df_comparison = pd.merge(df_budget, df_actuals, on='category', how='outer')
df_comparison['budget'] = df_comparison['budget'].fillna(0)
df_comparison['actual_spent'] = df_comparison['actual_spent'].fillna(0)
df_comparison['difference'] = df_comparison['actual_spent'] - df_comparison['budget']

df_comparison
```

---

## ✅ 6. Save Final Outputs

```python
df_cleaned.to_csv('output/cleaned_transactions.csv', index=False)
df_comparison.to_csv('output/budget_vs_actual.csv', index=False)
```

---

Now you're ready to run the entire pipeline from input → cleaning → categorization → budget analysis.
